In [135]:
# import libraries
from web3 import Web3
import os
from eth_account import Account
from pathlib import Path
from getpass import getpass
from constants import *
from pprint import pprint

import subprocess
import json

from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from eth_account import Account

from dotenv import load_dotenv


# loads environment variables without the need to reload jupyter lab
load_dotenv()

True

In [123]:
# Importing mnemonic from .env
mnemonic = os.getenv('MNEMONIC')

In [124]:
# Connecting to localhost network ETH/Geth
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [125]:
def derive_wallets(mnemonic, coin, numderive):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey,pubkeyhash,xprv,xpub --coin="{coin}" --numderive="{numderive}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [126]:
# testing derive_wallets function
derive_wallets(mnemonic, ETH, 3)

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x04B10D08a8586666bA03a8dbaddaF3a2babf17A9',
  'privkey': '0x119451ea6b0653675d2f02c17d726054db2bdcbbdb476d860a20d417edfae863',
  'pubkey': '02ee8c3f71cf1e7378bb5e89d6a799addc0cdfb2d44435160f55b7e062392f46ac',
  'pubkeyhash': 'a7643494fc9fb6448f5d11c8b87ff4cfd1ad30dd',
  'xprv': 'xprvA32TK2dTSdq2NXi8y5YqEFgEm9UdQzXaGRmtr3sDoskmfR9Gt1wBV8DqjbupWrNCjtb5FtcL6Nz66EcMwzVgout5YxFoTx9ckm1MDH8Z1qQ',
  'xpub': 'xpub6G1oiYAMH1PKb1nc575qbPcyKBK7pTFRdehVeSGqNDHkYDURRZFS2vYKatcuJP67aCmzbYGArNDVRRZ4WDeVFjp2NT3P1KWJUFyBBUTj8sr'},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0x527775c4b25b14721ad1dB9D190dA5a5d5aF3a91',
  'privkey': '0xf8cb43809f8d8d4d011e4f36447a251dd4d9dc1bb349d0d8f5110b14c5aac8e6',
  'pubkey': '037a54805b95cfded35cba7ca2be407b361d7577f0d58a9c16c173db3fa71e9533',
  'pubkeyhash': '6b718a5c0a96099ea6df4186526ec351d21a39ab',
  'xprv': 'xprvA32TK2dTSdq2QFLCpFrHYWLno7152iGdbigjuJQbTc1qsy94HZ731r6QqYVrD9twHK9gmjeXmo9b9h97KmAtyfmMinvA8yPBgABmCrKauo

In [137]:
# Create an object
coins = {'BTC': derive_wallets(mnemonic, BTC, 3), 'ETH': derive_wallets(mnemonic, ETH, 3), 'BTCTEST': derive_wallets(mnemonic, BTCTEST, 3)}

# Testing the object
pprint(coins)

{'BTC': [{'address': '1AoDRxW266rdH4F3aJ7djwGjmdjEDqzeKi',
          'path': "m/44'/0'/0'/0/0",
          'privkey': 'L5KMcLhqave1Bidyqx9GhiTeCG24FjtDPdnaCeeLa6CoBUPhRc9g',
          'pubkey': '021f1eaa32cf1cf4209a272b13fc72fd5ee3617d1ee51de82c0cca9920cff2d552',
          'pubkeyhash': '6b76a48030c9c17f089fc6fc2a6a43aaac9f7ab7',
          'xprv': 'xprvA2cUhHgDpuqsNhTRpRZK2xZHfX8ZmMEzCL1D2M2R69ooQbfRZaTeh9QzsB1J7UgqyXnrR3JEoLTWKj2Gvv9GJ56zWdydvY8eZbf27FDk9sY',
          'xpub': 'xpub6Fbq6oD7fHQAbBXtvT6KQ6W2DYy4AoxqZYvopjS2eVLnHPza77muEwjUiQSN314WxWfxyCRAWoHvkix3nRNur69dAEXZMSvEGsfJEBRd44o'},
         {'address': '1Na6mpcAFTNpbJ4mPKKt5aUa8FxEcyLYup',
          'path': "m/44'/0'/0'/0/1",
          'privkey': 'Kwwq9873MPrMukrm32311VnqSTmhQvXFQWSgN5AAAVjFJHw28aSx',
          'pubkey': '021f4d72ed2975da7f17e03985edb4c721136542c4bb86636af70034ca504716c1',
          'pubkeyhash': 'ec9d2fe38fb2fbb045fabf4edcb5ebe30e7ca3af',
          'xprv': 'xprvA2cUhHgDpuqsRqqHQZEiW3At2AaonNCkNFTHSEHxr7nNGBJW

In [128]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [129]:
priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 3)[0]['privkey'])

In [130]:
# Setting Ethereum Account to a variable
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 5)[0]['privkey'])
eth_acc

In [131]:
# Setting Bitcoin TestNet Account to a variable
btc_acc = priv_key_to_account(BTCTEST, derive_wallets(mnemonic, BTCTEST, 5)[0]['privkey'])
btc_acc

<PrivateKeyTestnet: n4ZJHGZ2g83NwjSPvnAHdsvEEP5nYcd7hM>

In [132]:
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({"from": eth_acc.address, "to": recipient, "value": amount})
        return {
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(btc_acc.address, [(recipient, amount, BTC)])

In [133]:
def send_tx(coin, account, recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        signed_txn = btc_acc.sign_transaction(txn)
        result = NetworkAPI.broadcast_tx_testnet(signed_txn)
        #print(result.hex())
        #return result.hex()

### Checking Balance

In [89]:
btctest_key_sender = wif_to_key("cUAbbJPPBkzokbkrikQJkpoKCNEoEoopDD2pS7q9eAMbCWimWZe5")

In [103]:
btctest_key_sender.get_balance('btc')

'0.02123504'

In [87]:
btctest_key_recipient = wif_to_key("cSxEKHxs3A25g8J6zTN9jEJTpXKAuNUU4jPbKoQEZWxC7w3WoRej")

In [121]:
btctest_key_recipient.get_balance('btc')

'0.01'

## Sending Transactions

### Ethereum Transaction

In [ ]:
create_tx(ETH, eth_acc, '0x527775c4b25b14721ad1dB9D190dA5a5d5aF3a91', )

In [ ]:
send_tx(ETH, eth_acc, '0x527775c4b25b14721ad1dB9D190dA5a5d5aF3a91', )

### Bitcoin TestNet Transaction

In [119]:
send_tx(BTCTEST, btc_acc, 'ms4mjqE7mghTeL2nrmLq9LaxzcP3KZHvE4', .005)